# electronic-gadget-identifier
Machine learning project that identifies different electronic gadgets

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms, models
from torch import nn, optim

In [ ]:
pretrained_size = 256
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds= [0.229, 0.224, 0.225]

transform = transforms.Compose([
                                transforms.Resize(pretrained_size),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = pretrained_means, std = pretrained_stds)
                              ])

In [ ]:
trainset = datasets.ImageFolder(r'datasets\training', transform=transform)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [ ]:
plt.imshow(images[1].permute(1, 2, 0).numpy().astype(np.uint8))

In [ ]:
figure = plt.figure()
num_of_images = 60
for index in range(1, num_of_images + 1):
    plt.subplot(6, 10, index)
    plt.axis('off')
    plt.imshow(images[index].permute(1, 2, 0).numpy().astype(np.uint8))

In [ ]:
import torchvision.models as models

pretrained_model = models.vgg16_bn(pretrained = True)

print(pretrained_model)

In [ ]:
classification_layer = nn.Sequential(nn.Linear(25088, 4096), 
                                     nn.ReLU(),
                                     nn.Dropout(p=0.3),
                                     nn.Linear(4096, 2),
                                     nn.LogSoftmax(dim=1))

pretrained_model.classifier =  classification_layer

print(pretrained_model.classifier)
model = pretrained_model

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
torch.set_num_threads(8)

In [ ]:
def run_conf_features(datasetloader):
    conv_features = []
    labels_list = []

    for images, labels in datasetloader:
        model.eval()
        with torch.no_grad():
            # modeling for each image batch
            conv_layer_output = model.features(images)        

            # flatten output as the next layers are the fully connected layers
            output = conv_layer_output.view(conv_layer_output.size(0), -1)

            conv_features.extend(output.data.cpu().numpy())
            labels_list.extend(labels.data.cpu().numpy())
            
    return (torch.Tensor(conv_features), torch.LongTensor(labels_list))

In [ ]:
conv_features, labels_list = run_conf_features(trainloader)

In [ ]:
conv_features_dataset = torch.utils.data.TensorDataset(conv_features, labels_list)
trainloader = torch.utils.data.DataLoader(conv_features_dataset, batch_size=64, shuffle=True)